In [ ]:
def moustache(s):
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

def data_cleanup_table(df):
    # Возвращает таблицу для EDA с анализом признаков
    
    # Создание таблицы
    columns = ['Признак', 'Тип_данных', 'Пропущенных_значений', 
           'Выбросов', 'Ошибочных_значений', 'Управление', 'Комментарий']
    table = pd.DataFrame(columns=columns)
    table['Признак'] = df.columns
    df_len = len(df)
    print('Создание таблицы...')
    #
    for sign in df.columns:
        table.loc[table['Признак'] == sign, 'Тип_данных'] = df[sign].dtype
    
    
    # Анализ пропущенных значений
    for sign in tqdm(df.columns, desc='Анализ пропущенных значений\t'):
        table.loc[table['Признак'] == sign, 'Пропущенных_значений'] = df[sign].isna().sum()
        if df[sign].isna().sum() / df_len > 0.2:
            table.loc[table['Признак'] == sign, 'Управление'] = 'Удалить'
            table.loc[table['Признак'] == sign, 'Комментарий'] = 'Пропусков больше 20% '
            
    
    # Анализ выбрасов
    for sign in tqdm(df.columns, desc='Анализ выбросов\t\t\t'):
        if df[sign].dtype == 'object':
            table.loc[table['Признак'] == sign, 'Выбросов'] = -1
        else:
            tmp = moustache(df[sign]) # возращает границы усов (а, б)
            table.loc[table['Признак'] == sign, 'Выбросов'] = df[(df[sign] < tmp[0]) |
                                                                            (df[sign] > tmp[1])][sign].count()      
    
    return table

In [41]:
# ПРИМЕР:    НЕ ЗАПУСКАТЬ!
table = data_cleanup_table(df_train)
table.head()

Создание таблицы...


Анализ выбросов			: 100%|██████████| 291/291 [00:02<00:00, 106.60it/s]


,Признак,Тип_данных,Пропущенных_значений,Выбросов,Ошибочных_значений,Управление,Комментарий
0,id,int64,0,0,NaN,NaN,NaN
1,timestamp,object,0,-1,NaN,NaN,NaN
2,full_sq,int64,0,963,NaN,NaN,NaN
3,life_sq,float64,6383,0,NaN,Удалить,Пропусков больше 20%
4,floor,float64,167,0,NaN,NaN,NaN


---
---